# **Extraccion datos de ligas europeas**

In [1]:
# Se importan librerias
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

In [2]:
# Lista de paises de ligas, esto para poder iterar en la pagina por pais
paises_ligas = ['England', 'Spain', 'Germany', 'Italy', 'France']

# Itera en la lista para extraer informacion por pais
for pais in paises_ligas:  

    # Se define esta variable para ejecutar el navegador en segundo plano
    opciones = Options()
    opciones.headless = True

    # Se define la pagina web y driver
    website = "https://www.adamchoi.co.uk/overs/detailed"
    path = "C:/Users/PERSONAL/Desktop/Data scince/UDEMY - WebScraping/chromedriver"

    # Definimos driver para navegar en la pagina web
    driver = webdriver.Chrome(path, options=opciones)
    driver.get(website)

    # Definimos boton donde dar click para que aparezcan todos los partidos
    todos_partidos_boton = driver.find_element_by_xpath('//label[@analytics-event="All matches"]')
    todos_partidos_boton.click()

    # Variable que nos permite seleccionar lista desplegable y escoger el pais de la liga
    lista_desplegable = Select(driver.find_element_by_id('country'))
    lista_desplegable.select_by_visible_text(pais)

    # Se define el un tiempo muerto en segundos para esperar los datos cargados de la pagina
    time.sleep(2)

    # Se extrae la data de partidos
    partidos = driver.find_elements_by_tag_name('tr')

    # Definimos listas de los datos del partido
    fecha = []
    equipo_local = []
    resultado = []
    equipo_visitante = []

    # Iteramos en partidos para extraer la data texto en las listas creadas
    for partido in partidos:
        fecha.append(partido.find_element_by_xpath('./td[1]').text)
        equipo_local.append(partido.find_element_by_xpath('./td[2]').text)
        resultado.append(partido.find_element_by_xpath('./td[3]').text)
        equipo_visitante.append(partido.find_element_by_xpath('./td[4]').text)

    # cerrar pagina
    driver.quit()

    # Creamos data_frame para los datos extraidos
    df_partidos_resultado = pd.DataFrame({'Fecha': fecha, 'Equipo_Local': equipo_local, 'Resultado': resultado, 
                                    'Equipo_Visitante': equipo_visitante})
    
    # Elimina partidos duplicados
    df_partidos_resultado = df_partidos_resultado.drop_duplicates()

    # Eliminamos los registros que traigan como Resulta ? - ?
    df_partidos_resultado = df_partidos_resultado.loc[df_partidos_resultado['Resultado'] != '? - ?']
    df_partidos_resultado = df_partidos_resultado.loc[df_partidos_resultado['Resultado'] != '-']

    # Creamos dos columnas a partir de la columna Resultado
    df_partidos_resultado[['Gol_Local', 'Gol_Visitante']] = df_partidos_resultado['Resultado'].str.split(' - ', expand=True)

    # Combertimos las dos columnas nuevas a tipo de dato int
    df_partidos_resultado['Gol_Local'] = df_partidos_resultado['Gol_Local'].astype(int)
    df_partidos_resultado['Gol_Visitante'] = df_partidos_resultado['Gol_Visitante'].astype(int)

    # Creamos la columna ambos marcan
    df_partidos_resultado['Ambos_Marcan'] = df_partidos_resultado.apply(lambda row: 1 if row['Gol_Local']>0 and row['Gol_Visitante']>0 else 0, axis=1)

    # Función para determinar el resultado del partido
    def determinar_resultado(gol_local, gol_visitante):
        if gol_local > gol_visitante:
            return 'G'
        elif gol_local < gol_visitante:
            return 'P'
        else:
            return 'E'

    # Aplicar la función a cada fila para determinar el resultado de cada equipo
    df_partidos_resultado['Resultado_Local'] = df_partidos_resultado.apply(lambda row: determinar_resultado(row['Gol_Local'], row['Gol_Visitante']), axis=1)
    df_partidos_resultado['Resultado_Visitante'] = df_partidos_resultado.apply(lambda row: determinar_resultado(row['Gol_Visitante'], row['Gol_Local']), axis=1)

    # Exporta datos extraidos con solo resultados local y visitante
    df_partidos_resultado.to_csv(f'{pais} Datos Resultados_Condicion.csv', index=False)

    # Saca copia de df_partidos_resultado para agregar columna de equipo 
    df_partidos_resultado_local = df_partidos_resultado.copy()
    df_partidos_resultado_local['Equipo'] = df_partidos_resultado['Equipo_Local']

    # Saca copia de df_partidos_resultado para agregar columna de equipo 
    df_partidos_resultado_visitante = df_partidos_resultado.copy()
    df_partidos_resultado_visitante['Equipo'] = df_partidos_resultado['Equipo_Visitante']

    # Concatena los df_partidos_resultado_local y df_partidos_resultado_visitante para tabla final
    df_partidos_resultado = pd.concat([df_partidos_resultado_local,df_partidos_resultado_visitante], ignore_index=True)

    # Crea columna nueva para saber el resultado del equipo
    df_partidos_resultado['Estado_Partido'] = df_partidos_resultado.apply(lambda row: row['Resultado_Local'] if row['Equipo']==row['Equipo_Local'] else row['Resultado_Visitante'], axis=1)

    # Exporta los datos ya extraidos y transformados en archivo csv
    df_partidos_resultado.to_csv(f'{pais} Datos Resultados.csv', index=False)

## **Corners**

In [2]:
# Lista de paises de ligas, esto para poder iterar en la pagina por pais
paises_ligas_corners = ['England', 'Spain', 'Germany', 'Italy', 'France']

# Itera en la lista para extraer informacion por pais
for pais in paises_ligas_corners:  

    # Se define esta variable para ejecutar el navegador en segundo plano
    opciones = Options()
    opciones.headless = True

    # Se define la pagina web y driver
    website = "https://www.adamchoi.co.uk/corners/detailed"
    path = "C:/Users/PERSONAL/Desktop/Data scince/UDEMY - WebScraping/chromedriver"

    # Definimos driver para navegar en la pagina web
    driver = webdriver.Chrome(path, options=opciones)
    driver.get(website)

    # Definimos boton donde dar click para que aparezcan todos los partidos
    todos_partidos_boton = driver.find_element_by_xpath('//label[@analytics-event="All matches"]')
    todos_partidos_boton.click()

    # Variable que nos permite seleccionar lista desplegable y escoger el pais de la liga
    lista_desplegable = Select(driver.find_element_by_id('country'))
    lista_desplegable.select_by_visible_text(pais)

    # Se define el un tiempo muerto en segundos para esperar los datos cargados de la pagina
    time.sleep(2)

    # Se extrae la data de partidos
    partidos = driver.find_elements_by_tag_name('tr')

    # Definimos listas de los datos del partido
    fecha = []
    equipo_local = []
    resultado = []
    equipo_visitante = []

    # Iteramos en partidos para extraer la data texto en las listas creadas
    for partido in partidos:
        fecha.append(partido.find_element_by_xpath('./td[1]').text)
        equipo_local.append(partido.find_element_by_xpath('./td[2]').text)
        resultado.append(partido.find_element_by_xpath('./td[3]').text)
        equipo_visitante.append(partido.find_element_by_xpath('./td[4]').text)

    # cerrar pagina
    driver.quit()

    # Creamos data_frame para los datos extraidos
    df_partidos_corners = pd.DataFrame({'Fecha': fecha, 'Equipo_Local': equipo_local, 'Resultado': resultado, 
                                    'Equipo_Visitante': equipo_visitante})
    
    # Elimina partidos duplicados
    df_partidos_corners = df_partidos_corners.drop_duplicates()

    # Eliminamos los registros que traigan como Resulta ? - ?
    df_partidos_corners = df_partidos_corners.loc[df_partidos_corners['Resultado'] != '? - ?']
    df_partidos_corners = df_partidos_corners.loc[df_partidos_corners['Resultado'] != '-']

    # Creamos dos columnas a partir de la columna Resultado
    df_partidos_corners[['Corners_Local', 'Corners_Visitante']] = df_partidos_corners['Resultado'].str.split(' - ', expand=True)

    # Combertimos las dos columnas nuevas a tipo de dato int
    df_partidos_corners['Corners_Local'] = df_partidos_corners['Corners_Local'].astype(int)
    df_partidos_corners['Corners_Visitante'] = df_partidos_corners['Corners_Visitante'].astype(int)

    # Función para determinar el resultado del partido
    def determinar_resultado(corners_local, corners_visitante):
        if corners_local > corners_visitante:
            return 'G'
        elif corners_local < corners_visitante:
            return 'P'
        else:
            return 'E'

    # Aplicar la función a cada fila para determinar el resultado de corners de cada equipo
    df_partidos_corners['Resultado_Local'] = df_partidos_corners.apply(lambda row: determinar_resultado(row['Corners_Local'], row['Corners_Visitante']), axis=1)
    df_partidos_corners['Resultado_Visitante'] = df_partidos_corners.apply(lambda row: determinar_resultado(row['Corners_Visitante'], row['Corners_Local']), axis=1)

    # Exporta datos extraidos con solo resultados local y visitante
    df_partidos_corners.to_csv(f'{pais} Datos Corners_Condicion.csv', index=False)

    # Saca copia de df corners para agregar columna de equipo 
    df_partidos_corners_local = df_partidos_corners.copy()
    df_partidos_corners_local['Equipo'] = df_partidos_corners['Equipo_Local']

    # Saca copia de df corners para agregar columna de equipo 
    df_partidos_corners_visitante = df_partidos_corners.copy()
    df_partidos_corners_visitante['Equipo'] = df_partidos_corners['Equipo_Visitante']

    # Concatena los df_partidos_corners_local y df_partidos_corners_visitante
    df_partidos_corners = pd.concat([df_partidos_corners_local,df_partidos_corners_visitante], ignore_index=True)

    # Crea columna nueva para saber el resultado del equipo
    df_partidos_corners['Estado_Partido'] = df_partidos_corners.apply(lambda row: row['Resultado_Local'] if row['Equipo']==row['Equipo_Local'] else row['Resultado_Visitante'], axis=1)

    # Exporta los datos ya extraidos y transformados en archivo csv
    df_partidos_corners.to_csv(f'{pais} Datos Corners.csv', index=False)